<a href="https://colab.research.google.com/github/shovitraj/GCloud/blob/main/VISIONAPI_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-cloud-vision
!pip install --upgrade google-cloud-storage


In [7]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving avid-atrium-296502-9e6921103dfe.json to avid-atrium-296502-9e6921103dfe (1).json
User uploaded file "avid-atrium-296502-9e6921103dfe.json" with length 2343 bytes


In [1]:
import os, io
# Imports Credential File:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "avid-atrium-296502-9e6921103dfe.json"
print("Service Account Key: {}".format(os.environ["GOOGLE_APPLICATION_CREDENTIALS"]))

#'gs://meropdf/Syllabus.pdf'

Service Account Key: avid-atrium-296502-9e6921103dfe.json


In [ ]:
# import re
# from google.cloud import vision
# from google.cloud import storage
# from google.protobuf import json_format

In [ ]:
client = vision.ImageAnnotatorClient

In [ ]:
# # Supported mime_types are: 'application/pdf' and 'image/tiff'
# mime_type = 'application/pdf'

# # How many pages should be grouped into each json output file.
# batch_size = 2

In [ ]:
# feature = vision.Feature(
#         type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

In [ ]:
# gcs_source_uri = 'gs://meropdf/Syllabus.pdf'
# gcs_source = vision.GcsSource(uri=gcs_source_uri)
# input_config = vision.InputConfig(gcs_source=gcs_source, mime_type=mime_type)

In [ ]:
# gcs_destination_uri='gs://meropdf/pdf_result '
# gcs_destination = vision.GcsDestination(uri=gcs_destination_uri)  
# output_config = vision.OutputConfig(gcs_destination=gcs_destination, batch_size=batch_size)

In [2]:
import re
import json
from google.cloud import vision
from google.cloud import storage
from google.protobuf import json_format

""" 
# pip install --upgrade google-cloud-storage
"""

client = vision.ImageAnnotatorClient()

batch_size = 2
mime_type = 'application/pdf'
feature = vision.Feature(
        type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

gcs_source_uri = 'gs://meropdf/Syllabus.pdf'
gcs_source = vision.GcsSource(uri=gcs_source_uri)
input_config = vision.InputConfig(gcs_source=gcs_source, mime_type=mime_type)

gcs_destination_uri = 'gs://meropdf/pdf_result'
gcs_destination = vision.GcsDestination(uri=gcs_destination_uri)
output_config = vision.OutputConfig(gcs_destination=gcs_destination, batch_size=batch_size)

async_request = vision.AsyncAnnotateFileRequest(
    features=[feature], input_config=input_config, output_config=output_config)

operation = client.async_batch_annotate_files(requests=[async_request])
operation.result(timeout=180)

storage_client = storage.Client()
match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
bucket_name = match.group(1)
prefix = match.group(2)
bucket = storage_client.get_bucket(bucket_name)

# List object with the given prefix
blob_list = list(bucket.list_blobs(prefix=prefix))
print('Output files:')
for blob in blob_list:
    print(blob.name)

output = blob_list[0]
json_string = output.download_as_string()

response = json.loads(json_string)
# response = json_format.Parse(
#             json_string, vision.AnnotateFileResponse())

# The actual response for the first page of the input file.
first_page_response = response['responses'][0]
annotation = first_page_response['fullTextAnnotation']


print('Full text:\n')
print(annotation['text'])

Output files:
pdf_resultoutput-1-to-2.json
pdf_resultoutput-3-to-4.json
pdf_resultoutput-5-to-6.json
Full text:

PHYS 225- Fundamental Physics, Section 05(31825)
Instructor
Shovit Bhari Office Phone, Email Office hour Zoom Meetings
Course Information
Synchronous
Asynchronous
Supplemental Instructor (SI) SI Session Email:
Textbook
CANVAS
Exam dates*
Physics 225- Fundamental Physics: Mechanics Fall 2020
WEB Fully instructions online (657) 278 2148, sbhari@fullerton.edu Mo We 7:00 pm – 8:00 pm (Zoom) Mo We 5:30 pm – 6:45 pm
Approximately 80% of the lectures will be conducted virtually on Zoom. There will we instructional activities where you are required to attend the Zoom session.
You will not be required to be present during your regular meeting times during asynchronous lectures. I will post the video lectures on the Canvas. Asynchronous meetings, which I will announce in advance. However, you are required to complete assignments and quizzes if they are due on those days.
Kyle Tanovitz

In [8]:
second_page_response = response['responses'][1]
annotation1 = second_page_response['fullTextAnnotation']

In [6]:
print('Full text:\n')
print(annotation1['text'])

Full text:

PHYS 225- Fundamental Physics, Section 05(31825)
Course objective, learning goals, and prerequisites
In this course, you will learn about some topics in fundamental physics. Physics is an approach to learning about the world around us (and beyond) through measurement. It is happening all around us, and I encourage you to look for examples of how physics is involved in our daily experiences. In this course, we will primarily have focused on the branch of physics called mechanics, which is concerned with motion: how and why things move. You will learn physical concepts, how these fundamental concepts apply to real-world situations, and how to use the physical concepts you will learn to solve problems. You will also understand the physical concepts you learn in the corequisite lab course, Physics 225L.
In this course, we won’t just use mathematics to talk about physical ideas-many of the ideas themselves are mathematical. So, to master these concepts (and thus to succeed in th